<a href="https://colab.research.google.com/github/laylaelbaaly/NeuralStyleTransfer/blob/main/IOB_NST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IOB-NST Algorithm based on the origial paper “A Neural Algorithm of Artistic Style” by Gatys et al.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ===========================
# IMPORT REQUIRED LIBRARIES
# ===========================
import os
import numpy as np
import time
from keras.preprocessing.image import load_img, img_to_array
# from keras.applications import vgg19
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from tensorflow.keras.models import Model
K = tf.keras.backend
from scipy.optimize import fmin_l_bfgs_b
from PIL import Image

# ==============================
# GLOBAL SETTINGS AND CONSTANTS
# ==============================

img_height = 512
result_prefix = "results"

# Weighting factors for each loss term
# content_weight = 0.025
# style_weight = 1.0
# total_variation_weight = 1e-4

# Define dataset directories

# Set content and style image folders in Google Drive
content_dir = '/content/drive/MyDrive/University of London/Year 3/FINAL PROJECT/Final Project Code/IOB-NST/content'
style_dir = '/content/drive/MyDrive/University of London/Year 3/FINAL PROJECT/Final Project Code/IOB-NST/style'

# Output folder (where stylized images will be saved)
output_dir = '/content/drive/MyDrive/University of London/Year 3/FINAL PROJECT/Final Project Code/IOB-NST/stylized_outputs'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# ===================================
# IMAGE PROCESSING UTILITY FUNCTIONS
# ===================================

def preprocess_image(image_path, target_size):
    """
    Loads and preprocesses an image for VGG19.
    """
    img = load_img(image_path, target_size=target_size)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return vgg19.preprocess_input(img)

def deprocess_image(x):
    """
    Reverses VGG19 preprocessing to return a displayable image.
    """
    x = x.reshape((img_height, img_width, 3))
    x[:, :, 0] += 103.939  # Add mean RGB values (reverse normalization)
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]      # Convert BGR to RGB
    x = np.clip(x, 0, 255).astype('uint8')
    return Image.fromarray(x)

In [ ]:
# ==============================
# LOSS FUNCTION DEFINITIONS
# ==============================

def content_loss(base, combination):
    """
    Measures content similarity (high-level feature maps).
    """
    return K.sum(K.square(combination - base))

def gram_matrix(x):
    """
    Computes the Gram matrix (feature correlation matrix).
    """
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    return K.dot(features, K.transpose(features))

def style_loss(style, combination):
    """
    Measures style similarity via Gram matrix difference.
    """
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

def total_variation_loss(x):
    """
    Encourages spatial smoothness in the generated image.
    """
    a = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, 1:, :img_width-1, :])
    b = K.square(x[:, :img_height-1, :img_width-1, :] - x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [ ]:
# ===================================
# EVALUATOR: WRAPS LOSS AND GRADIENT
# ===================================

class Evaluator(object):
    """
    Used to interface with L-BFGS optimizer: returns loss and gradient separately,
    while caching them to avoid recomputation.
    """
    def __init__(self, fetch_loss_and_grads_fn):
        self.fetch_loss_and_grads = fetch_loss_and_grads_fn
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        """
        Returns the loss value and caches the gradients.
        """
        x = x.reshape((1, img_height, img_width, 3))
        outs = self.fetch_loss_and_grads([x])
        self.loss_value = outs[0]
        self.grads_values = outs[1].flatten().astype('float64')
        return self.loss_value

    def grads(self, x):
        """
        Returns cached gradients from the last loss computation.
        """
        grads = np.copy(self.grads_values)
        self.loss_value = None
        self.grads_values = None
        return grads

In [ ]:
# ===================================
# NEURAL STYLE TRANSFER FUNCTION
# ===================================

def run_style_transfer(content_path, style_path, output_path,
                       content_weight, style_weight, total_variation_weight,
                       iterations=300):

    global img_width
    width, height = load_img(content_path).size
    img_width = int(width * img_height / height)

    # Preprocess content and style images
    content_image = preprocess_image(content_path, (img_height, img_width))
    style_image = preprocess_image(style_path, (img_height, img_width))

    # Initialize the combination image as a trainable variable
    combination_image = tf.Variable(content_image, dtype=tf.float32)

    # Load VGG19 and freeze layers
    vgg = vgg19.VGG19(weights='imagenet', include_top=False)
    vgg.trainable = False

    # Layers to extract features from
    content_layer = 'block5_conv2'
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    output_layers = [content_layer] + style_layers

    # Build model that outputs selected layer activations
    outputs = [vgg.get_layer(name).output for name in output_layers]
    feature_extractor = Model(inputs=vgg.input, outputs=outputs)

    # Extract features for content and style images
    content_features = feature_extractor(content_image)[0]
    style_features = feature_extractor(style_image)[1:]

    # Updated Gram matrix computation for 4D tensors
    def gram_matrix(tensor):
        """
        Computes the Gram matrix from a 4D feature map.
        Input shape: (1, height, width, channels)
        Output shape: (1, channels, channels)
        """
        assert len(tensor.shape) == 4, "Input must be 4D (batch, h, w, c)"
        batch_size, height, width, channels = tensor.shape
        x = tf.reshape(tensor, [batch_size, height * width, channels])
        gram = tf.matmul(x, x, transpose_a=True)
        return gram / tf.cast(height * width, tf.float32)

    # Loss function + gradient computation
    def compute_loss_and_grads():
        with tf.GradientTape() as tape:
            combo_outputs = feature_extractor(combination_image)
            combo_content = combo_outputs[0]
            combo_styles = combo_outputs[1:]

            # Content loss: how much the content changes
            c_loss = content_weight * tf.reduce_mean(tf.square(combo_content - content_features))

            # Style loss: how much the textures match

            # s_loss = 0
            # for style_feat, combo_feat in zip(style_features, combo_styles):
            #     S = gram_matrix(style_feat)
            #     C = gram_matrix(combo_feat)
            #     channels = 3
            #     size = img_height * img_width
            #     s_loss += tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))
            # s_loss *= style_weight / len(style_layers)

            s_loss = 0
            for style_feat, combo_feat in zip(style_features, combo_styles):
                S = gram_matrix(style_feat)
                C = gram_matrix(combo_feat)
                shape = tf.shape(combo_feat)
                channels = tf.cast(shape[-1], tf.float32)
                size = tf.cast(shape[1] * shape[2], tf.float32)
                s_loss += tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))
            s_loss *= style_weight / len(style_layers)


            # Total variation loss: encourages spatial smoothness
            tv_loss = total_variation_weight * total_variation_loss(combination_image)

            # Final combined loss
            total_loss = c_loss + s_loss + tv_loss

        grads = tape.gradient(total_loss, combination_image)
        return total_loss, grads

    # Use Adam optimizer
    optimizer = tf.optimizers.Adam(learning_rate=10.0)

    # Optimization loop
    for i in range(iterations):
        loss, grads = compute_loss_and_grads()
        optimizer.apply_gradients([(grads, combination_image)])
        print(f"Iteration {i+1}, Loss: {loss.numpy():,.2f}")

    # Save the stylized image
    final_img = deprocess_image(combination_image.numpy())
    final_img.save(output_path)
    print(f"Saved: {output_path}")

In [ ]:
# ===================================
# MAIN LOOP: APPLY TO ALL IMAGE PAIRS
# ===================================

# # Sort and match files from content and style directories
# content_files = sorted([f for f in os.listdir(content_dir) if f.lower().endswith(('.jpg', '.png'))])
# style_files = sorted([f for f in os.listdir(style_dir) if f.lower().endswith(('.jpg', '.png'))])

# # Apply NST to each content-style image pair
# for content_file, style_file in zip(content_files, style_files):
#     content_path = os.path.join(content_dir, content_file)
#     style_path = os.path.join(style_dir, style_file)
#     output_path = os.path.join(output_dir, f"stylized_{content_file}_{style_file}")

#     print(f"\nProcessing: {content_file} + {style_file}")
#     run_style_transfer(content_path, style_path, output_path, 1e2, 1e5, 1e-6, 300)

# Sort all valid image files
content_files = sorted([f for f in os.listdir(content_dir) if f.lower().endswith(('.jpg', '.png'))])
style_files = sorted([f for f in os.listdir(style_dir) if f.lower().endswith(('.jpg', '.png'))])

# Apply each style to each content image
for content_file in content_files:
    for style_file in style_files:
        content_path = os.path.join(content_dir, content_file)
        style_path = os.path.join(style_dir, style_file)

        # Create a clear output filename
        output_name = f"stylized_{content_file}_with_{style_file}"
        output_path = os.path.join(output_dir, output_name)

        print(f"\nProcessing: {content_file} + {style_file}")
        run_style_transfer(content_path, style_path, output_path, 1e2, 1e5, 1e-6, 300)


Processing: modern-residential-building-beautiful-recreation-area-modern-residential-building-recreation-area-131403554.jpg + 83316.jpg
Iteration 1, Loss: 7,610,693.00
Iteration 2, Loss: 3,187,717.00
Iteration 3, Loss: 3,205,992.75
Iteration 4, Loss: 2,346,490.00
Iteration 5, Loss: 1,622,154.00
Iteration 6, Loss: 1,696,455.12
Iteration 7, Loss: 1,225,491.12
Iteration 8, Loss: 1,247,440.12
Iteration 9, Loss: 1,020,615.25
Iteration 10, Loss: 1,016,860.56
Iteration 11, Loss: 845,503.62
Iteration 12, Loss: 792,300.81
Iteration 13, Loss: 739,252.31
Iteration 14, Loss: 655,652.88
Iteration 15, Loss: 653,057.88
Iteration 16, Loss: 562,113.19
Iteration 17, Loss: 560,567.12
Iteration 18, Loss: 498,650.84
Iteration 19, Loss: 485,397.69
Iteration 20, Loss: 456,224.28
Iteration 21, Loss: 425,749.88
Iteration 22, Loss: 414,362.66
Iteration 23, Loss: 382,797.94
Iteration 24, Loss: 378,824.31
Iteration 25, Loss: 349,196.03
Iteration 26, Loss: 345,321.75
Iteration 27, Loss: 321,883.84
Iteration 28, L

In [ ]:
#image is not styled enough - fix!!
#display content, style, and result
#error graphs
#surveys

#think about common final dataset for portfolio (check jing review and notes)